# **Manejo de DataSet de vinos para predicción de la calidad**

<a href="https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009">**DataSet Kaggle**</a>

1. [Obtención de datos](#obtención-de-datos)

2. [Lectura de datos y analisis de ellos](#lectura-de-datos-y-analisis-de-ellos)
    1. [Picos](#picos)
     <p></p>

3. [Visualización de datos](#visualización-de-datos)
    1. [Scatter](#scatter-de-ácido-volátil-ácido-citrico-y-calidad-del-vino)

    2. [BoxPlot](#boxplot-de-coorrelaciones)

    3. [Mátriz de coorrelación](#matriz-de-correlación-y-mapa-de-calor)
    
    4. [Scatter Matrix](#scatter-matrix)

4. [Experimentación con columnas](#experimentación-con-columnas)

5. [Preparación de datos](#preparación-de-datos)
    1. [Limpieza](#limpieza)
    
    2. [División de datos](#división-de-datos)

6. [Definición de los modelos](#definición-de-los-modelos)
    1. [LINEAL](#lineal)

    2. [ÁRBOL DE DECISIONES](#árbol-de-decisiones)

    3. [VALIDACIÓN CRUZADA DEL ÁRBOL DE DECISIONES](#validación-cruzada-del-árbol-de-decisiones)

    4. [RANDOM FOREST MODELO](#random-forest-modelo)

    5. [SVR](#svr)

    6. [XGBOOST](#xgboost)

# **Obtención de datos**

Descargamos el dataset de forma local y cogemos su ruta para leer el .csv y meterlo a un DataFrame de Pandas donde trataremos los datos

In [ ]:
import pathlib

path=str(pathlib.Path().resolve())+"/"

In [ ]:
import pandas as pd


wine = pd.read_csv(path+"winequality-red.csv")

# **Lectura de datos y analisis de ellos**

Leemos los datos y analizamos que tipo de datos son las columnas y vemos cuales son Nulos. Etc...

In [ ]:
wine

In [ ]:
wine.info()

<p style="color:yellow">ADVERTENCIA</p>Como vemos no tenemos ninguna columna con valores nulos, en caso de tenerlo deberiamos tratarlo en su respectivo tratamiento de datos

In [ ]:
wine.head()

In [ ]:
wine.describe()

## Picos

Vemos los picos, donde muchos componen picos normales, pero otros picos son más extraños

In [ ]:
wine.hist(bins=20,figsize=(10,10))

# **Visualización de datos**

## Scatter de ácido volátil, ácido citrico y calidad del vino

Vemos una pequeña relación entre el ácido volatil con el citrico, cuando menos acido cítrico por norma general es mayor el ácido volatil, mientras que si se acerca el vino al pico del ácido volatil, pierde calidad

In [ ]:
wine.plot.scatter(y="volatile acidity",x="citric acid",alpha=0.2,c="quality",cmap="Reds")

## BoxPlot de coorrelaciones

Tenemos un BoxPlot de las coorrelaciones de la calidad del vino con las diferentes columnas que ya hay en la propia tabla, esto nos permite eliminar los Ouliers que eliminaremos más adelante

In [ ]:
from matplotlib import pyplot as plt

from Clases.Matplot.BoxPlot import BoxPlot


BoxPlot.box_plot(types=wine.columns,by="quality",dataFrame=wine,deepColor="deeppink",faceColor="Pink",color="Pink",ballsColor="deeppink")


## Matriz de correlación y mapa de calor

Con el mapa de calor de las relaciones podemos los campos que más se relacionan entre ellos y los que son más importantes para calidad así permitiendo descartar columnas no muy importantes

In [ ]:
from Clases.Matplot.HeatMap import HeatMap


corr_matrix = wine.corr()
HeatMap.heat_map(corr_matrix,corr_matrix.columns,corr_matrix,corr_matrix.columns,corr_matrix.columns,cmap="YlGn",figsize=(10,10),weight="bold",textColor="red")

## Scatter matrix

Scatter matrix de las columnas para ver los pícos de las columnas y diferencias entre relaciones

In [ ]:
from pandas.plotting import scatter_matrix
columns = ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol","quality"]
scatter_matrix(wine[columns], figsize=(12,8))

# **Experimentación con columnas**

En este caso me interesó probar con datos que me parecen que pueden tener relación entre ellos en este caso 
<li style="color:red">El total del sulfuro con su free sulfur</li>
<li style="color:red">El ácido fixed y el cítrico</li>
<li style="color:green">El ácido fixed y la densidad</li>

In [ ]:
wine['free_sulfur_dioxide_per_total_sulfur_dioxide'] = wine['total sulfur dioxide']/wine['free sulfur dioxide']

wine['citric_acid_per_fixed_acidity'] = wine['fixed acidity']/wine['citric acid']

wine['density_per_fixed_acidity'] = wine['fixed acidity']/wine['density']

wine['acidity'] = wine['fixed acidity'] + wine['volatile acidity'] + wine['citric acid']

In [ ]:
wine.corr()

Vemos que dos de estas no salen con buenas relaciones, entonces los descartamos

In [ ]:
wine.drop(['citric_acid_per_fixed_acidity',
           #'free_sulfur_dioxide_per_total_sulfur_dioxide',
           ],axis=1,inplace=True)



# **Preparación de datos**

## Limpieza

Vemos que los campos no tienen nulos por lo que no será necesario tratar los valores nulos de estos

In [ ]:
wine.info()

Definimos una función para el tratamiento de Outliers

In [ ]:
import numpy as np
from scipy import stats


def tratamiento_outliers(columna,x,y):
    """Min y Max de una columna

    Args:
        columna (object): Columna a tratar
        x (float): Min
        y (float): Max

    Returns:
        List(float): Min y Max
    """
    sorted(columna)
    Q1,Q3 = np.percentile(columna,[x,y])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return lower_range,upper_range

def borrar_outliers(df,columns):
    """Borra los Outliers

    Args:
        df (DataFrame): DataFrame con los datos
        columns (X): Columnas

    Returns:
        DataFrame: Nuevo DataFrame con los datos borrados
    """
    for i,value in columns.items():
        if df[i].dtype == 'float64':
            low,high=tratamiento_outliers(df[i],value[0],value[1])
            df.drop(
                df[(df[i] > high) | (df[i] < low) ].index , 
                inplace=True)
            #df_filtrado = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
    return df

In [ ]:
#columnas=["quality","alcohol","sulphates","chlorides","residual sugar","citric acid","fixed acidity"]
#columnas = ["quality","alcohol","sulphates","pH","density","total sulfur dioxide","free sulfur dioxide","chlorides","residual sugar","citric acid","volatile acidity","fixed acidity"]
from matplotlib import axis


s = wine['quality']
wine.drop('quality',axis=1,inplace=True)
columnas = {#'citric acid':[10,90],
            'chlorides':[10.5,89.5],
            #'density':[17.5,82.5],
            #'alcohol':[10,90], 
            'free_sulfur_dioxide_per_total_sulfur_dioxide':[10,90],
            'sulphates':[2,98], 
            'free sulfur dioxide':[5,95],
            }
wine=borrar_outliers(wine,columnas)
wine['quality'] = s
columnas = list(wine.columns)
#columnas.remove('residual sugar')
#columnas.remove('free sulfur dioxide')
#columnas.remove('pH')
columnas

In [ ]:
wine.size

In [ ]:
BoxPlot.box_plot(types=columnas,by="quality",dataFrame=wine,deepColor="deeppink",faceColor="Pink",color="Pink",ballsColor="deeppink")

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler(feature_range=(0,1))


#wine['fixed acidity'] = scaler.fit_transform(wine[['fixed acidity']])
#wine['volatile acidity'] = scaler.fit_transform(wine[['volatile acidity']])
#wine['citric acid'] = scaler.fit_transform(wine[['citric acid']])


## División de datos

Definimos una función para dividir el entrenamiento con un ratio (Ya existe una función que lo hace por si solo)

In [ ]:
def split_train_test(data,test_ratio):
    # indices = [i for i in range(len(data))]; indices = np.shuffle(.....)
  # NOS DA UNA LSITA BARAJADA
  indices = np.random.permutation(len(data))
  # LE DECIMOS CUANTO TEST SE USARÁ
  lg_test = int(len(data) * test_ratio)
  # SE REPARTEN
  test_indices = indices[:lg_test]
  train_indices = indices[lg_test:]
  # Y SE DEVUELVE UNA TABLA DE ENTRENAMIENTO Y OTRA DE TEST
  return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
np.random.seed(42)
x_train,x_test = split_train_test(wine,0.2)

Definimos las variables x_train y y_train que serán los datos para entrenar el modelo

In [ ]:
x_train = x_train.loc[:,columnas]
x_test = x_test.loc[:,columnas]
#y_train = dt_train["quality"].copy()
#x_train = dt_train.drop(["quality"],axis=1)

y_train = x_train["quality"].copy()
x_train = x_train.drop(["quality"],axis=1)

y_test = x_test["quality"].copy()
x_test = x_test.drop(["quality"],axis=1)



### Mejores X Columnas

In [ ]:
#from sklearn.feature_selection import SelectKBest,mutual_info_classif
#
#print(x_train.shape)
#selector = SelectKBest(mutual_info_classif,k=14)
#selector.fit(x_train,y_train)
#x_train = selector.transform(x_train)
#x_test = selector.transform(x_test)


### Estandar Scaler

In [ ]:
from sklearn.discriminant_analysis import StandardScaler


scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# Mejora un poco el lineal y otros modelos pero el SVR por mucho

### Polinomios

In [ ]:
#from sklearn.preprocessing import PolynomialFeatures
#
#
#poly = PolynomialFeatures(degree=1)
#poly.fit(x_train)
#x_train = poly.transform(x_train)
#x_test = poly.transform(x_test)
#x_train

# **Definición de los modelos**

## LINEAL

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(x_train,y_train)

# MÓDELO CREADO

In [ ]:
datos_prueba =  x_train[:10]
print("PREDICCIONES: ",lin_reg.predict(datos_prueba))
y_reales = y_train[:10]
print("Reales: ", list(y_reales))

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

predicciones = lin_reg.predict(x_test)
mse = mean_squared_error(y_test, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
score = r2_score(y_test, predicciones)
# No lo realiza tan mal
print(f"mae(ERROR MEDIO ABSOLUTO): {mae}   mse(ERROR CUADRÁTICO): {mse}  score:{score}")
# 0.4341950674596512


Vemos que tiene un Score bastante alto para ser un modelo lineal

### VALIDACIÓN CRUZADA LINEAL

In [ ]:
from sklearn.model_selection import cross_val_score


lin_score = cross_val_score(lin_reg, x_train, y_train,
                            scoring = "neg_mean_squared_error", cv=10)
root_lin_score = np.sqrt(-lin_score)
print("Scores: ", root_lin_score)
print("Media: ", root_lin_score.mean())
print("Desviación Std", root_lin_score.std())

## ÁRBOL DE DECISIONES

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(x_train, y_train)

In [ ]:
datos_prueba =  x_train
p = tree_reg.predict(datos_prueba)
#print("PREDICCIONES: ",p)
y_reales = y_train
#print("Reales: ", list(y_reales))
for index,i in enumerate(p):
    if i != list(y_reales)[index]:
        print(f"EN LA POSICIÓN {index} ES DIFERENTE: \nEL REAL: {list(y_reales)[index]} -- LA PREDICCIÓN: {i}")

In [ ]:
predicciones = tree_reg.predict(x_train)
mse = mean_squared_error(y_train, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_train, predicciones)
score = r2_score(y_train, predicciones)
print(f"mae: {mae}   rmse: {mse} r2_score: {score}")

In [ ]:
predicciones = tree_reg.predict(x_test)
mse = mean_squared_error(y_test, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
score = r2_score(y_test, predicciones)
print(f"mae: {mae}   rmse: {mse} r2_score: {score}")

Tiene un Score de 1.0 Pero engañoso, en el momento que pongamos un dato fuera del DataSet este fallará

### VALIDACIÓN CRUZADA DEL ÁRBOL DE DECISIONES

In [ ]:
from sklearn.model_selection import cross_val_score

lin_score = cross_val_score(tree_reg, x_train, y_train,
                            scoring = "neg_mean_squared_error", cv=10)
root_lin_score = np.sqrt(-lin_score)
print("Scores: ", root_lin_score)
print("Media: ", root_lin_score.mean())
print("Desviación Std", root_lin_score.std())

Vemos que este modelo tiene bastante buena media en el modelo

## RANDOM FOREST MODELO

*<p style="color:yellow">(DE LOS MÁS UTILIZADOS, ÁRBOL QUE FUNCIONA CORRECTAMENTE Y MÁS SIMPLE)</p>*

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100)

rf_reg.fit(x_train, y_train)

In [ ]:
predicciones = rf_reg.predict(x_test)
mse = mean_squared_error(y_test, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
score = r2_score(y_test, predicciones)

# DE LOS MAYORES SCORES QUE TIENE -1 - 1
print(f"mae: {mae}   rmse: {mse} r2_score: {score}")

In [ ]:
datos_prueba =  x_test[:20]
p = rf_reg.predict(datos_prueba)
print("PREDICCIONES: ",p)
y_reales = y_test[:20]
print("Reales: ", list(y_reales))

Vemos que las mediciones son más precisas con un Score mucho más alto

In [ ]:
# VALIDACIÓN DE TIPO CROSS AL RANDOM TREE FOREST

rf_score = cross_val_score(rf_reg, x_train, y_train,
                            scoring = "neg_mean_squared_error", cv=10)
root_lin_score = np.sqrt(-rf_score)
print("RF cross")
print("Scores: ", root_lin_score)
print("Media: ", root_lin_score.mean())
print("Desviación Std", root_lin_score.std())

## SVR

*<p style="color:yellow">(DE LO MÁS UTILIZADO)</p>*

In [ ]:
from sklearn.svm import SVR

sv_reg = SVR()

sv_reg.fit(x_train, y_train)

In [ ]:
predicciones = sv_reg.predict(x_test)
mse = mean_squared_error(y_test, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
score = r2_score(y_test, predicciones)
print(f"mae: {mae}   rmse: {mse} r2_score: {score}")
#0.5982260797614003

In [ ]:
datos_prueba =  x_test[:20]
p = sv_reg.predict(datos_prueba)
print("PREDICCIONES: ",p)
y_reales = y_test[:20]
print("Reales: ", list(y_reales))

In [ ]:
svg_score = cross_val_score(sv_reg, x_train, y_train,
                            scoring = "neg_mean_squared_error", cv=10)
root_lin_score = np.sqrt(-svg_score)
print("SV cross")
print("Scores: ", root_lin_score)
print("Media: ", root_lin_score.mean())
print("Desviación Std", root_lin_score.std())

Vemos que la media es decente y su Score tmb pero hay modelos con mejores resultados como el Random Forest

# **XGBOOST**

*(EL MÁS UTILIZADO ACTUALMENTE Y MUY EXTACTO)*

In [ ]:
#!conda install -y -c conda-forge xgboost

In [ ]:
import xgboost as xgb

xgb_reg = xgb.XGBRegressor(objetive="reg:squarederror")

#x_train['quality_cat'] = x_train['quality_cat'].astype(int)
xgb_reg.fit(x_train, y_train)

In [ ]:
predicciones = xgb_reg.predict(x_test)
mse = mean_squared_error(y_test, predicciones)
mse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
score = r2_score(y_test, predicciones)
print(f"mae: {mae}   rmse: {mse} r2_score: {score}")
#0.5193044214733533

In [ ]:
datos_prueba =  x_test[:20]
p = xgb_reg.predict(datos_prueba)
print("PREDICCIONES: ",p)
y_reales = y_test[:20]
print("Reales: ", list(y_reales))

In [ ]:
xgb_score = cross_val_score(xgb_reg, x_train, y_train,
                            scoring = "neg_mean_squared_error", cv=10)
root_lin_score = np.sqrt(-xgb_score)
print("XGB cross")
print("Scores: ", root_lin_score)
print("Media: ", root_lin_score.mean())
print("Desviación Std", root_lin_score.std())

Vemos que esta es el que mejores resultados dá, dando casi un Score de 1 siendo e máximo, manteniendo una media de casi 0.6

# **Evaluación del modelo con el set**

El resultado más parecido al deseado seria con el Random Forest teniendo un rendimiento bastante alto y acercandose a los datos de test con los que no fue entrenado